In [1]:
import requests
import json

# 気象庁データの取得
# jma_url = "https://www.jma.go.jp/bosai/forecast/data/forecast/130000.json"
jma_url = "https://weather.tsukumijima.net/api/forecast/city/130010"
jma_json = requests.get(jma_url).json()

# 取得したいデータを選ぶ
# jma_date = jma_json[0]["timeSeries"][0]["timeDefines"][0]
# jma_weather = jma_json[0]["timeSeries"][0]["areas"][0]["weathers"][0]
# jma_rainfall = jma_json["Feature"][0]["Property"]["WeatherList"]["Weather"][0]["Rainfall"]
# # 全角スペースの削除
# jma_weather = jma_weather.replace('　', '')

# print(jma_date)
# print(jma_weather)
# print(jma_rainfall)

In [2]:
jma_json

{'publicTime': '2025-05-06T05:00:00+09:00',
 'publicTimeFormatted': '2025/05/06 05:00:00',
 'publishingOffice': '気象庁',
 'title': '東京都 東京 の天気',
 'link': 'https://www.jma.go.jp/bosai/forecast/#area_type=offices&area_code=130000',
 'description': {'publicTime': '2025-05-06T04:34:00+09:00',
  'publicTimeFormatted': '2025/05/06 04:34:00',
  'headlineText': '',
  'bodyText': '\u3000東シナ海には低気圧があって、東へ進んでいます。また、関東甲信地方は湿った空気の影響を受けています。\n\n\u3000東京地方は、曇りで雨の降っている所があります。\n\n\u3000６日は、低気圧が日本の南から関東の東へ進む見込みです。このため、曇りで、昼前から夕方は雨となるでしょう。伊豆諸島では、雨で雷を伴い激しく降る所がある見込みです。\n\n\u3000７日は、高気圧に覆われますが、寒気や気圧の谷の影響を受ける見込みです。このため、曇り時々晴れで、朝晩は雨の降る所があるでしょう。\n\n【関東甲信地方】\n\u3000関東甲信地方は、雨や曇りとなっています。\n\n\u3000６日は、低気圧が日本の南から関東の東へ進む見込みです。このため、雨や曇りとなり、雷を伴い激しく降る所があるでしょう。\n\n\u3000７日は、高気圧に覆われますが、寒気や気圧の谷の影響を受ける見込みです。このため、曇りや晴れで、雨の降る所があるでしょう。\n\n\u3000関東地方と伊豆諸島の海上では、うねりを伴い、６日はしけとなり、７日は波が高いでしょう。船舶は高波に注意してください。',
  'text': '\u3000東シナ海には低気圧があって、東へ進んでいます。また、関東甲信地方は湿った空気の影響を受けています。\n\n\u3000東京地方は、曇りで雨の降っている所があります。\n\n\u3000６日は、低気圧が日本の南から関東の

In [3]:
import pandas as pd

weather_df = pd.read_csv('../data/weather_data.csv', encoding="shift-jis", skiprows=[0,1,2,4,5])

In [ ]:

weather_df = weather_df[["年月日", "最高気温(℃)", "最低気温(℃)", "天気概況(昼：06時〜18時)"]]
weather_df = weather_df.rename(columns={
    "年月日": "date",
    "最高気温(℃)": "max_temp",
    "最低気温(℃)": "min_temp",
    "天気概況(昼：06時〜18時)": "weather",
})

In [5]:
weather_df["date"] = pd.to_datetime(weather_df["date"], format="%Y/%m/%d")

In [6]:
import  zipfile
import os

In [7]:
zip_dir = os.path.expanduser('../data/power_usage')
result = []

In [8]:
for zip_name in sorted(os.listdir(zip_dir)):
    if not zip_name.endswith('.zip'):
        continue
    
    zip_path = os.path.join(zip_dir, zip_name)
    
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        for csv_filename in zip_ref.namelist():
            
            if not csv_filename.endswith('.csv'):
                continue
            
            with zip_ref.open(csv_filename) as csv_file:
                try:
                    df = pd.read_csv(csv_file, encoding="shift-jis", skiprows=54)
                    max_power = df["当日実績(５分間隔値)(万kW)"].max()
                    result.append({
                        "date": csv_filename.split('_')[0],
                        "max_power": max_power
                    })
                except Exception as e:
                    print(f"Error reading {csv_filename}: {e}")
                    

In [9]:
power_usage_df = pd.DataFrame(result)
power_usage_df["date"] = pd.to_datetime(power_usage_df["date"], format="%Y%m%d")

In [10]:
integrated_df = pd.merge(weather_df, power_usage_df, on="date", how="inner")

In [11]:
integrated_df

,date,max_temp,min_temp,weather,avg_temp,max_power
0,2022-04-01,12.4,3.9,曇一時雨,7.6,3877
1,2022-04-02,12.9,3.6,薄曇時々晴,8.5,3351
2,2022-04-03,10.1,6.6,雨一時曇,8.2,3546
3,2022-04-04,9.8,6.7,大雨,8.5,4185
4,2022-04-05,17.9,8.2,曇一時雨後晴,12.6,3663
...,...,...,...,...,...,...
1001,2024-12-27,12.7,3.8,晴後一時曇,7.4,3789
1002,2024-12-28,11.5,1.4,晴後一時曇,5.4,3622
1003,2024-12-29,12.3,0.5,快晴,6.0,3476
1004,2024-12-30,10.3,3.5,薄曇一時晴,6.2,3466
